In [3]:
from pathlib import Path
from statistics import mode
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np


import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

from model import CowNet
from utils import dataset, binary_acc

dataDir = Path.cwd().parent.parent/'Data/processed'
cow_dataset = pd.read_csv(dataDir/'cow_dataset.csv')
data_x = cow_dataset.iloc[:, :12]
data_y = cow_dataset.iloc[:, 12]

# set up hyper-parameters
learning_rate = 0.0005
epoch = 250
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.autograd.set_detect_anomaly(True) 

# split the dataset into train and test data
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2, random_state=50)
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
train_data = dataset(X_train, y_train)
test_data = dataset(X_test, y_test)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

# initiate model for training
model = CowNet(input_size=X_train.shape[1])
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.BCEWithLogitsLoss()

model.train()
for e in range(1, epoch+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        X_batch.float()
        y_batch.float()
        y_pred = model(X_batch)

        loss = loss_fn(y_pred, y_batch.reshape(-1, 1))
        acc = binary_acc(y_pred, y_batch.reshape(-1, 1))

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    
        
    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')



Epoch 001: | Loss: 0.53641 | Acc: 0.736
Epoch 002: | Loss: 0.46746 | Acc: 0.780
Epoch 003: | Loss: 0.45811 | Acc: 0.783
Epoch 004: | Loss: 0.44623 | Acc: 0.788
Epoch 005: | Loss: 0.43136 | Acc: 0.789
Epoch 006: | Loss: 0.42404 | Acc: 0.793
Epoch 007: | Loss: 0.40203 | Acc: 0.808
Epoch 008: | Loss: 0.44235 | Acc: 0.786
Epoch 009: | Loss: 0.43270 | Acc: 0.785
Epoch 010: | Loss: 0.40691 | Acc: 0.800
Epoch 011: | Loss: 0.41552 | Acc: 0.797
Epoch 012: | Loss: 0.40797 | Acc: 0.798
Epoch 013: | Loss: 0.39935 | Acc: 0.801
Epoch 014: | Loss: 0.38470 | Acc: 0.812
Epoch 015: | Loss: 0.40129 | Acc: 0.805
Epoch 016: | Loss: 0.40949 | Acc: 0.804
Epoch 017: | Loss: 0.37300 | Acc: 0.818
Epoch 018: | Loss: 0.36570 | Acc: 0.825
Epoch 019: | Loss: 0.37407 | Acc: 0.822
Epoch 020: | Loss: 0.35821 | Acc: 0.829
Epoch 021: | Loss: 0.36129 | Acc: 0.830
Epoch 022: | Loss: 0.35993 | Acc: 0.831
Epoch 023: | Loss: 0.34783 | Acc: 0.836
Epoch 024: | Loss: 0.34660 | Acc: 0.839
Epoch 025: | Loss: 0.35474 | Acc: 0.833


In [9]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

from model import CowNet
from utils import dataset, binary_acc

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# load test dataset
modelDir = Path.cwd()
dataDir = Path.cwd().parent.parent/'Data/processed'
cow_dataset = pd.read_csv(dataDir/'cow_test_dataset.csv')
data_x = cow_dataset.iloc[:, :12]
data_y = cow_dataset.iloc[:, 12]
X_test = data_x.to_numpy()
y_test = data_y.to_numpy()
test_data = dataset(X_test, y_test)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

# load the model from the load dir
model = CowNet(input_size=X_test.shape[1])
model.load_state_dict(torch.load(modelDir/'best_model.pt'))
model.to(device)

y_pred_list = []
model.eval()
test_acc = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        y_test_pred = model(X_batch)
        #y_test_pred = torch.sigmoid(y_test_pred)
        #y_pred_label = torch.round(y_test_pred)
        #y_pred_list.append(y_pred_label.cpu().numpy())
        acc = binary_acc(y_test_pred, y_batch.reshape(-1, 1))
        test_acc += acc.item()
    test_acc = test_acc/len(test_loader)
print(f'\nTest Accuracy: {test_acc:.3f}')
#y_pred_list = [a.squeeze().tolist() for a in y_pred_list]
#confusion_matrix(y_test, y_pred_list)
#print(classification_report(y_test, y_pred_list))


Test Accuracy: 0.943


In [8]:
cow_dataset

,0,1,2,3,4,5,6,7,8,9,10,11,0.1
0,0.435798,0.673838,0.022405,0.174142,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1
1,0.634241,0.637752,0.000301,0.664908,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1
2,0.529183,0.342817,0.000473,0.266491,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1
3,0.186770,0.349757,0.069493,0.279683,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0
4,0.750973,0.320264,0.000129,0.026385,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4681,0.272374,0.132547,0.006967,0.675462,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1
4682,0.793774,0.427828,0.001118,0.316623,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0
4683,0.575875,0.277585,0.040423,0.482850,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0
4684,0.603113,0.322693,0.000516,0.110818,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0


In [4]:
test_loader


In [21]:
from torch import Tensor

A = Tensor(1, 3, 3)

In [22]:
A

tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]])

In [23]:
n_node = 3
n_edge_types = 2
A_in = A[:, :, :6]
A_out = A[:, :, 6:]

In [24]:
A_out

tensor([], size=(1, 3, 0))

In [25]:
A_in

tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]])